In [50]:
import requests
import pandas as pd
from datetime import datetime

In [58]:
CLIENT_ID = '1249cb8b97e04bfc8b4de2b1e505993c'
CLIENT_SECRET = '1112a627856942f1b859af480957a140'
PLAYLIST_ID = '2YRe7HRKNRvXdJBp9nXFza'
import requests


# Declare the global variable
track_info_list = []

def get_access_token():
    response = requests.post('https://accounts.spotify.com/api/token', 
                             data={'grant_type': 'client_credentials'}, 
                             auth=(CLIENT_ID, CLIENT_SECRET))
    return response.json()['access_token']

def get_audio_features(track_ids, token):
    audio_features = []
    for i in range(0, len(track_ids), 100):  # Process in batches of 100
        batch_ids = track_ids[i:i + 100]
        url = f'https://api.spotify.com/v1/audio-features?ids={",".join(batch_ids)}'
        response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
        if response.status_code == 200:
            audio_features.extend(response.json()['audio_features'])
        else:
            print("Error fetching audio features:", response.status_code)
            return []
    return audio_features

def get_artist_info(artist_id, token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching artist info:", response.status_code)
        return {}

def get_album_info(album_id, token):
    url = f'https://api.spotify.com/v1/albums/{album_id}'
    response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching album info:", response.status_code)
        return {}

def get_playlist_tracks(playlist_id, token):
    global track_info_list  # Declare the use of the global variable
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    track_ids = []

    while url:
        response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
        if response.status_code == 200:
            data = response.json()
            for item in data['items']:
                track = item['track']
                if track:
                    album = track['album']
                    artist_info = get_artist_info(track['artists'][0]['id'], token)
                    genre = get_album_info(album['id'], token).get('genres', [])
                    
                    track_info = {
                        'name': track['name'],
                        'artist': ', '.join(artist['name'] for artist in track['artists']),
                        'popularity': track['popularity'],
                        'id': track['id'],  # Store track ID for fetching audio features
                        'release_date': album['release_date'],
                        'track_number': track['track_number'],
                        'explicit': track['explicit'],
                        'duration_ms': track['duration_ms'],  # Duration in milliseconds
                        'artist_popularity': artist_info.get('popularity', 0),
                    }
                    track_info_list.append(track_info)  # Append to the global variable
                    track_ids.append(track['id'])  # Collect track IDs for audio features
            url = data['next']  # Get the URL for the next page
        else:
            print("Error fetching playlist tracks:", response.status_code)
            break

    # Get audio features after fetching all tracks
    audio_features = get_audio_features(track_ids, token)
    for i, features in enumerate(audio_features):
        if features:  # Only add features if they were successfully retrieved
            track_info_list[i].update({
                'danceability': features['danceability'],
                'energy': features['energy'],
                'key': features['key'],
                'loudness': features['loudness'],
                'mode': features['mode'],
                'speechiness': features['speechiness'],
                'acousticness': features['acousticness'],
                'instrumentalness': features['instrumentalness'],
                'liveness': features['liveness'],
                'valence': features['valence'],
                'tempo': features['tempo'],
            })

# Main execution
if __name__ == "__main__":
    token = get_access_token()
    get_playlist_tracks(PLAYLIST_ID, token)
    for track in track_info_list:
        print(track)  # Print all track information, including audio features


Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album info: 429
Error fetching artist info: 429
Error fetching album in

In [59]:
df = pd.DataFrame(track_info_list)

In [60]:
df['explicit_num'] = df['explicit'].astype(int)
df = df.drop(columns=['explicit'])

In [61]:
#Will be important to remember to have a way to describe what each column means. For example, what is key 1.0? 
df.head(5)


,name,artist,popularity,id,release_date,track_number,duration_ms,artist_popularity,explicit_num
0,Blinding Lights,The Weeknd,89,0VjIjW4GlUZAMYd2vXMi3b,2020-03-20,9,200040,95,0
1,Shape of You,Ed Sheeran,85,7qiZfU4dY1lWllzX7mPBI3,2017-03-03,4,233712,88,0
2,Someone You Loved,Lewis Capaldi,87,7qEHsqek33rTcFNT9PFqLf,2019-05-17,4,182160,80,0
3,As It Was,Harry Styles,89,4Dvkj6JhhA12EX05fT7y2e,2022-05-20,4,167303,84,0
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",81,3KkXRkHbMCARz0aVfEt68P,2018-12-14,2,158040,92,0


genre
[]    896
Name: count, dtype: int64

In [39]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['id'])
df_copy.head(4)

,name,artist,popularity,release_date,track_number,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,explicit_num
0,Blinding Lights,The Weeknd,89,2020-03-20,9,200040,0.513,0.730,1.0,-5.940,1.0,0.0598,0.00143,0.000095,0.0897,0.334,171.001,0
1,Shape of You,Ed Sheeran,85,2017-03-03,4,233712,0.825,0.652,1.0,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,0
2,Someone You Loved,Lewis Capaldi,87,2019-05-17,4,182160,0.501,0.405,1.0,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,0
3,As It Was,Harry Styles,89,2022-05-20,4,167303,0.520,0.731,6.0,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,0


In [62]:
df_copy.head(5)

,name,artist,popularity,release_date,track_number,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,explicit_num
0,Blinding Lights,The Weeknd,89,2020-03-20,9,200040,0.513,0.730,1.0,-5.940,1.0,0.0598,0.00143,0.000095,0.0897,0.334,171.001,0
1,Shape of You,Ed Sheeran,85,2017-03-03,4,233712,0.825,0.652,1.0,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,0
2,Someone You Loved,Lewis Capaldi,87,2019-05-17,4,182160,0.501,0.405,1.0,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,0
3,As It Was,Harry Styles,89,2022-05-20,4,167303,0.520,0.731,6.0,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,0
4,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",81,2018-12-14,2,158040,0.760,0.478,2.0,-5.574,1.0,0.0467,0.55200,0.000000,0.0703,0.913,89.908,0
